# Fragment Class Demo

This notebook demonstrates the usage of the Fragment class in libfrag, which extends the Molecule class with fragment-specific functionality using the composite design pattern.

In [ ]:
# Import the libfrag module
import libfrag
import numpy as np

## Creating Basic Fragments

Let's start by creating some basic fragments with atoms and bonds.

In [ ]:
# Create atoms for a methane molecule
carbon = libfrag.Atom(6, 0.0, 0.0, 0.0)  # Carbon at origin
h1 = libfrag.Atom(1, 1.1, 0.0, 0.0)     # Hydrogen
h2 = libfrag.Atom(1, -0.5, 0.9, 0.0)    # Hydrogen
h3 = libfrag.Atom(1, -0.5, -0.9, 0.0)   # Hydrogen
h4 = libfrag.Atom(1, 0.0, 0.0, 1.1)     # Hydrogen

methane_atoms = [carbon, h1, h2, h3, h4]

print(f"Created {len(methane_atoms)} atoms for methane")
for i, atom in enumerate(methane_atoms):
    print(f"  Atom {i}: {atom.symbol} at ({atom.x:.1f}, {atom.y:.1f}, {atom.z:.1f})")

In [ ]:
# Create bond connectivity for methane (C-H bonds)
bond_connectivity = [(0, 1), (0, 2), (0, 3), (0, 4)]  # Carbon bonded to all hydrogens

# Create the fragment
methane_fragment = libfrag.Fragment(methane_atoms, bond_connectivity, "methane_frag")

print(f"Created fragment '{methane_fragment.fragment_id}'")
print(f"  Atoms: {methane_fragment.atom_count}")
print(f"  Bonds: {methane_fragment.bond_count}")
print(f"  Generation level: {methane_fragment.generation_level}")
print(f"  Is root fragment: {methane_fragment.is_root_fragment()}")

## Fragment Properties and Composition

In [ ]:
# Calculate fragment properties
properties = methane_fragment.calculate_fragment_properties()

print("Fragment Properties:")
for prop_name, prop_value in properties.items():
    print(f"  {prop_name}: {prop_value}")

In [ ]:
# Get fragment composition (element counts)
composition = methane_fragment.fragment_composition()

print("Fragment Composition:")
for element, count in composition.items():
    print(f"  {element}: {count}")

# Calculate molecular mass
print(f"\\nMolecular mass: {methane_fragment.molecular_mass:.2f} amu\")

## Creating Complex Molecules for Fragmentation

Let's create a larger molecule with multiple disconnected components to demonstrate fragmentation.

In [ ]:
# Create a molecule with methane + water (disconnected)
complex_atoms = [
    # Methane part
    libfrag.Atom(6, 0.0, 0.0, 0.0),    # Carbon
    libfrag.Atom(1, 1.1, 0.0, 0.0),    # H1
    libfrag.Atom(1, -0.5, 0.9, 0.0),   # H2
    libfrag.Atom(1, -0.5, -0.9, 0.0),  # H3
    
    # Water part (disconnected from methane)
    libfrag.Atom(8, 5.0, 0.0, 0.0),    # Oxygen
    libfrag.Atom(1, 5.5, 0.8, 0.0),    # H4
    libfrag.Atom(1, 5.5, -0.8, 0.0),   # H5
]

complex_bonds = [
    (0, 1), (0, 2), (0, 3),  # Methane bonds
    (4, 5), (4, 6)           # Water bonds
]

complex_fragment = libfrag.Fragment(complex_atoms, complex_bonds, "complex_molecule")

print(f"Created complex fragment with {complex_fragment.atom_count()} atoms and {complex_fragment.bond_count()} bonds")
print(f"Is connected: {complex_fragment.is_connected()}")

## Fragment Splitting by Connectivity

Now let's split the complex molecule into subfragments based on connectivity.

In [ ]:
# Fragment by connectivity (should separate methane and water)
subfragments = complex_fragment.fragment_by_connectivity()

print(f"Fragmentation created {len(subfragments)} subfragments:")
print(f"Parent fragment now has {complex_fragment.subfragment_count()} direct subfragments")

for i, subfrag in enumerate(subfragments):
    composition = subfrag.fragment_composition()
    comp_str = ", ".join([f"{elem}{count}" for elem, count in composition.items()])
    
    print(f"  Subfragment {i}: ID='{subfrag.fragment_id()}'")
    print(f"    Atoms: {subfrag.atom_count()}, Bonds: {subfrag.bond_count()}")
    print(f"    Composition: {comp_str}")
    print(f"    Generation level: {subfrag.generation_level()}")

## Fragment Tree Visualization

In [ ]:
# Display the fragment tree structure
print("Fragment Tree Structure:")
print(complex_fragment.to_tree_string())

## Inter-Fragment Links

Fragments can be linked to other fragments to represent covalent bonds that cross fragment boundaries.

In [ ]:
if len(subfragments) >= 2:
    # Create a link between the first two subfragments
    frag1_id = subfragments[0].fragment_id
    frag2_id = subfragments[1].fragment_id
    
    # Create a hypothetical bond between atom 0 of frag1 and atom 0 of frag2
    fragment_link = libfrag.FragmentLink(
        frag1_id, frag2_id,
        0, 0,  # Connect first atoms of each fragment
        libfrag.BondType.SINGLE,
        1.0  # Bond order
    )
    
    # Add the link to the first fragment
    subfragments[0].add_fragment_link(fragment_link)
    
    print(f"Added link: {fragment_link.to_string()}")
    print(f"Fragment '{frag1_id}' is now linked to: {list(subfragments[0].linked_fragment_ids)}")
    print(f"Is '{frag1_id}' linked to '{frag2_id}'? {subfragments[0].is_linked_to(frag2_id)}")

## Fragment Factory Methods

The Fragment class provides static factory methods for convenient creation.

In [ ]:
# Create a simple molecule first
simple_molecule = libfrag.Molecule(methane_atoms)

# Use factory method to create fragment from molecule
fragment_from_molecule = libfrag.Fragment.create_from_molecule(simple_molecule, "factory_test")

print(f"Created fragment from molecule: '{fragment_from_molecule.fragment_id()}'")
print(f"  Atoms: {fragment_from_molecule.atom_count()}")
print(f"  Is root: {fragment_from_molecule.is_root_fragment()}")

In [ ]:
# Create hierarchical fragmentation with size limit
large_molecule = libfrag.Molecule(complex_atoms)
hierarchical_fragment = libfrag.Fragment.create_fragment_hierarchy(
    large_molecule, 
    max_fragment_size=3,  # Max 3 atoms per fragment
    id_prefix="hierarchy"
)

print(f"Created hierarchical fragment: '{hierarchical_fragment.fragment_id()}'")
print(f"  Total atoms: {hierarchical_fragment.atom_count()}")
print(f"  Direct subfragments: {hierarchical_fragment.subfragment_count()}")
print(f"  Total atoms in tree: {hierarchical_fragment.total_atom_count()}")

print("\nHierarchical structure:")
print(hierarchical_fragment.to_tree_string())

## Advanced Fragmentation Methods

The Fragment class supports different fragmentation strategies.

In [ ]:
# Fragment by size (divide into roughly equal pieces)
size_fragment = libfrag.Fragment(complex_atoms, complex_bonds, "size_test")
size_subfragments = size_fragment.fragment_by_size(target_fragment_count=3)

print(f"Size-based fragmentation created {len(size_subfragments)} subfragments:")
for i, subfrag in enumerate(size_subfragments):
    print(f"  Subfragment {i}: {subfrag.atom_count()} atoms")

In [ ]:
# Custom fragmentation function
def custom_fragmenter(fragment):
    """Custom function that groups atoms by element type"""
    carbon_atoms = []
    hydrogen_atoms = []
    other_atoms = []
    
    for i in range(fragment.atom_count()):
        atom = fragment.atom(i)
        if atom.atomic_number() == 6:  # Carbon
            carbon_atoms.append(i)
        elif atom.atomic_number() == 1:  # Hydrogen
            hydrogen_atoms.append(i)
        else:
            other_atoms.append(i)
    
    groups = []
    if carbon_atoms:
        groups.append(carbon_atoms)
    if hydrogen_atoms:
        groups.append(hydrogen_atoms)
    if other_atoms:
        groups.append(other_atoms)
    
    return groups

# Apply custom fragmentation
custom_fragment = libfrag.Fragment(complex_atoms, complex_bonds, "custom_test")
custom_subfragments = custom_fragment.fragment_by_custom_function(custom_fragmenter)

print(f"Custom fragmentation created {len(custom_subfragments)} subfragments:")
for i, subfrag in enumerate(custom_subfragments):
    composition = subfrag.fragment_composition()
    comp_str = ", ".join([f"{elem}{count}" for elem, count in composition.items()])
    print(f"  Subfragment {i}: {comp_str}")

## Fragment Analysis and Comparison

In [ ]:
# Generate fragment hashes for comparison
hash1 = methane_fragment.fragment_hash()
hash2 = fragment_from_molecule.fragment_hash()

print(f"Methane fragment hash: {hash1}")
print(f"Factory fragment hash: {hash2}")
print(f"Hashes match: {hash1 == hash2}")

# Compare fragments
print(f"\nFragments are equal: {methane_fragment == fragment_from_molecule}")

In [ ]:
# Get detailed fragment information
print("Detailed fragment information:")
print(methane_fragment.to_fragment_string())

## Summary

The Fragment class provides:

1. **Inheritance from Molecule**: All molecular functionality is available
2. **Composite Pattern**: Hierarchical fragment organization with parent-child relationships
3. **Fragment Links**: Management of inter-fragment covalent bonds
4. **Multiple Fragmentation Strategies**: By connectivity, size, bond breaking, or custom functions
5. **Fragment Analysis**: Properties calculation, composition analysis, and comparison tools
6. **Factory Methods**: Convenient creation from molecules with automatic fragmentation

This makes it ideal for molecular fragmentation analysis in quantum chemistry applications.